In [5]:
import datetime
from IPython.core.pylabtools import figsize
from dateutil.relativedelta import relativedelta
from matplotlib import pylab
%pylab inline
%matplotlib inline
import numpy as np
import pandas as pd
from scipy import stats
import statsmodels.api as sm
import statsmodels
import matplotlib.pyplot as plt
import warnings
from itertools import product
import seaborn as sns
from scipy.linalg._solve_toeplitz import LinAlgError
from tqdm.notebook import tqdm

Populating the interactive namespace from numpy and matplotlib


c:\my files\python\fns_russia_rosstat\lib\site-packages\IPython\core\magics\pylab.py:159: UserWarning: pylab import has clobbered these variables: ['product']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [2]:
data = pd.read_csv('data/train.csv', sep=';')
data = data.drop(['region', 'okato'], axis=1)

In [ ]:
data.info()

In [3]:
data = data[::-1]

In [ ]:
list(data.oktmo.unique())

In [6]:
region_datas = []
for oktmo in sorted(data.oktmo.unique()):
    region_datas.append(pd.read_csv('data/region_data/{}_train.csv'.format(oktmo), index_col=['date'],
                                    parse_dates=['date'], dayfirst=True, sep=';'))

    for column in region_datas[-1].columns[1:]:
        region_datas[-1][column] = [x.replace(',', '.').replace('\xa0', '') if type(x) == str else x for x in region_datas[-1][column]]
        region_datas[-1][column] = region_datas[-1][column].astype(float)

In [ ]:
region_datas[20]

In [7]:
def invboxcox(y,lmbda):
   if lmbda == 0:
      return(np.exp(y))
   else:
      return(np.exp(np.log(lmbda*y+1)/lmbda))

## Preprocessing

In [8]:
for i, region in enumerate(region_datas):
    for column in region.columns.values[1:]:
        if sum(region[region[column] < 0][column]) != 0:
            region[column] = region[column].apply(lambda x: 0 if x < 0 else x)

        if sum(region[column][-60:] == 0) / 60 > 0.5:
            region[column] = 0.

        if len(region[region[column] == 0][column])  / len(region) < 0.55:
            region[column] = region[column].replace(0, np.nan).ffill().bfill()

        if sum(region[column][-60:] == 0) / 60 <= 0.5:
            region[column] = region[column].apply(lambda x: np.mean(region[region[column] != 0][column]) if x == 0 else x)

        if len(region[region[column] == 0][column])  / len(region) >= 0.90:
            region[column] = 0.


## Tuning модель с рядами Фурье

In [9]:
d =  0
D = 1

all_parameters = pd.read_csv('data/all_parameters.csv')

In [10]:
warnings.filterwarnings('ignore')

for i, region in tqdm(enumerate(region_datas), total=len(region_datas)):

    date_list = [datetime.datetime.strptime("01.04.2021", "%d.%m.%Y") +
                 relativedelta(days=x) for x in range(0,91)]
    future = pd.DataFrame(index=date_list, columns= region_datas[i].columns)
    region_datas[i] = pd.concat([region_datas[i], future])

    region_datas[i]['sin365_2'] = np.sin(2 * np.pi * region_datas[i].index.dayofyear / 365.25)
    region_datas[i]['cos365_2'] = np.cos(2 * np.pi * region_datas[i].index.dayofyear / 365.25)
    region_datas[i]['sin365_4'] = np.sin(4 * np.pi * region_datas[i].index.dayofyear / 365.25)
    region_datas[i]['cos365_4'] = np.cos(4 * np.pi * region_datas[i].index.dayofyear / 365.25)
    region_datas[i]['sin365_1'] = np.sin(np.pi * region_datas[i].index.dayofyear / 365.25)
    region_datas[i]['cos365_1'] = np.cos(np.pi * region_datas[i].index.dayofyear / 365.25)

    for column in region.columns[1:]:

        flag_not_boxcox = False

        if np.any(region[column] <= 0) or np.all(region[column] == region[column][0]):
            box = region_datas[i][column][:-91].copy()
            flag_not_boxcox = True
        else:
            box, lmbda = stats.boxcox(region_datas[i][column][:-91])

            if np.all([round(x, 5) for x in box] == round(box[0], 5)):
                lmbda = 0
                box = stats.boxcox(region_datas[i][column][:-91], lmbda)
                if np.all([round(x, 5) for x in box] == round(box[0], 5)):
                    box = region_datas[i][column][:-91].copy()
                    flag_not_boxcox = True

        if np.all(region[column] == 0.):
            region_datas[i]["{}_model".format(column)] = 0.
        else:
            p = int(all_parameters.loc[(all_parameters['column'] == column) & (all_parameters['oktmo'] == region_datas[i]['oktmo'][0])]['p'])
            P = int(all_parameters.loc[(all_parameters['column'] == column) & (all_parameters['oktmo'] == region_datas[i]['oktmo'][0])]['P'])
            if p == -1 or P == -1:
                p = 0
                P = 0

            model = sm.tsa.statespace.SARIMAX(box, exog=region_datas[i][['sin365_2', 'cos365_2', 'sin365_4', 'cos365_4']][:-91],
                                               order=(p, d, 0), seasonal_order=(P, D, 0, 7)).fit(disp=-1)

            if flag_not_boxcox:
                region_datas[i]["{}_model".format(column)] = model.predict(exog=region_datas[i][['sin365_2', 'cos365_2', 'sin365_4', 'cos365_4']][-91:],
                                                                            start=0, end=911)
            else:
                region_datas[i]["{}_model".format(column)] = invboxcox(model.predict(exog=region_datas[i][['sin365_2', 'cos365_2', 'sin365_4', 'cos365_4']][-91:],
                                                                                      start=0, end=911), lmbda)

            region_datas[i]["{}_model".format(column)] = region_datas[i]["{}_model".format(column)].replace(np.nan, -1)

            if column == 'potatoes':
                model = sm.tsa.statespace.SARIMAX(box, exog=region_datas[i][['cos365_1', 'sin365_4', 'cos365_4']][:-91],
                                               order=(p, d, 0), seasonal_order=(P, D, 0, 7)).fit(disp=-1)

                if flag_not_boxcox:
                    region_datas[i]["{}_model".format(column)] = model.predict(exog=region_datas[i][['cos365_1', 'sin365_4', 'cos365_4']][-91:],
                                                                            start=0, end=911)
                else:
                    region_datas[i]["{}_model".format(column)] = invboxcox(model.predict(exog=region_datas[i][['cos365_1', 'sin365_4', 'cos365_4']][-91:],
                                                                                      start=0, end=911), lmbda)


            if -1. in list(region_datas[i]["{}_model".format(column)][-91:]) or 5 * region_datas[i]["{}_model".format(column)][-91:].mean() < np.max(region_datas[i]["{}_model".format(column)][-91:]):

                model = sm.tsa.statespace.SARIMAX(box, trend='ct', order=(p, d, 0), seasonal_order=(P, D, 0, 7)).fit(disp=-1)

                if flag_not_boxcox:
                    region_datas[i]["{}_model".format(column)] = model.predict(start=0, end=911)
                else:
                    region_datas[i]["{}_model".format(column)] = invboxcox(model.predict(start=0, end=911), lmbda)

            if (i, column) == (1, 'spice') or (i, column) == (65, 'fruit'):
                model = sm.tsa.statespace.SARIMAX(box, order=(p, d, 0), seasonal_order=(P, D, 2, 28)).fit(disp=-1)

                if flag_not_boxcox:
                    region_datas[i]["{}_model".format(column)] = model.predict(start=0, end=911)
                else:
                    region_datas[i]["{}_model".format(column)] = invboxcox(model.predict(start=0, end=911), lmbda)

                region_datas[i]["{}_model".format(column)] = region_datas[i]["{}_model".format(column)].apply(lambda x: round(x * 1.03, 2))

            if (i, column) == (2, 'spice') or (i, column) == (55, 'spice') or (i, column) == (28, 'chicken'):
                model = sm.tsa.statespace.SARIMAX(box, exog=region_datas[i][['sin365_2', 'cos365_2', 'sin365_4', 'cos365_4']][:-91],
                                               order=(p, d, 1), seasonal_order=(0, D, 0, 7)).fit(disp=-1)

                if flag_not_boxcox:
                    region_datas[i]["{}_model".format(column)] = model.predict(exog=region_datas[i][['sin365_2', 'cos365_2', 'sin365_4', 'cos365_4']][-91:],
                                                                            start=0, end=911)
                else:
                    region_datas[i]["{}_model".format(column)] = invboxcox(model.predict(exog=region_datas[i][['sin365_2', 'cos365_2', 'sin365_4', 'cos365_4']][-91:],
                                                                                      start=0, end=911), lmbda)
            if (i, column) == (0, 'fruit'):
                model = sm.tsa.statespace.SARIMAX(box, exog=region_datas[i][['sin365_2', 'cos365_2', 'sin365_4', 'cos365_4']][:-91],
                                               order=(p, d, 1), seasonal_order=(0, D, 1, 7)).fit(disp=-1)

                if flag_not_boxcox:
                    region_datas[i]["{}_model".format(column)] = model.predict(exog=region_datas[i][['sin365_2', 'cos365_2', 'sin365_4', 'cos365_4']][-91:],
                                                                            start=0, end=911)
                else:
                    region_datas[i]["{}_model".format(column)] = invboxcox(model.predict(exog=region_datas[i][['sin365_2', 'cos365_2', 'sin365_4', 'cos365_4']][-91:],
                                                                                      start=0, end=911), lmbda)

            if (i, column) == (40, 'ai98_value') or (i, column) == (49, 'dt_value') or (i, column) == (75, 'margarine') \
                    or (i, column) == (75, 'margarine_value'):
                model = sm.tsa.statespace.SARIMAX(box, order=(p, d, 1), seasonal_order=(P, D, 0, 7)).fit(disp=-1)

                if flag_not_boxcox:
                    region_datas[i]["{}_model".format(column)] = model.predict(start=0, end=911)
                else:
                    region_datas[i]["{}_model".format(column)] = invboxcox(model.predict(start=0, end=911), lmbda)

            if (i, column) == (43, 'fruit'):
                region_datas[i]["{}_model".format(column)] = region_datas[i]["{}_model".format(column)].apply(lambda x: round(x * 0.96, 2))

            if (i, column) == (52, 'oil')  or (i, column) == (61, 'fruit') or (i, column) == (76, 'cpi_3') or (i , column) == (84, 'vegetables'):
                model = sm.tsa.statespace.SARIMAX(box, trend='t', order=(p, d, 1), seasonal_order=(P, D, 0, 7)).fit(disp=-1)

                if flag_not_boxcox:
                    region_datas[i]["{}_model".format(column)] = model.predict(start=0, end=911)
                else:
                    region_datas[i]["{}_model".format(column)] = invboxcox(model.predict(start=0, end=911), lmbda)

            if (i, column) == (64, 'salt_value') or (i, column) == (65, 'ai98_value'):
                model = sm.tsa.statespace.SARIMAX(box, order=(p, d, 1)).fit(disp=-1)

                if flag_not_boxcox:
                    region_datas[i]["{}_model".format(column)] = model.predict(start=0, end=911)
                else:
                    region_datas[i]["{}_model".format(column)] = invboxcox(model.predict(start=0, end=911), lmbda)

            if (i, column) == (64, 'margarine'):
                model = sm.tsa.statespace.SARIMAX(box, trend='t', order=(p, d, 1), seasonal_order=(P, D, 0, 7)).fit(disp=-1)

                if flag_not_boxcox:
                    region_datas[i]["{}_model".format(column)] = model.predict(start=0, end=911)
                else:
                    region_datas[i]["{}_model".format(column)] = invboxcox(model.predict(start=0, end=911), lmbda)

            if (i, column) == (65, 'margarine')  or (i, column) == (1, 'biscuits'):
                model = sm.tsa.statespace.SARIMAX(box, trend='ct', order=(p, d, 0), seasonal_order=(P, D, 1, 30)).fit(disp=-1)

                if flag_not_boxcox:
                    region_datas[i]["{}_model".format(column)] = model.predict(start=0, end=911)
                else:
                    region_datas[i]["{}_model".format(column)] = invboxcox(model.predict(start=0, end=911), lmbda)

        with open('logs.txt', 'a') as out:
            out.write("{}, {}\n".format(region_datas[i]['oktmo'][0], column))

        #plt.figure(figsize(15,7))
        #region_datas[i][column].plot()
        #region_datas[i]["{}_model".format(column)][8:].plot(color='r')
        #plt.ylabel("{}, {}".format(region['oktmo'][0], column))
        #pylab.show()

warnings.filterwarnings('default')

  0%|          | 0/85 [00:00<?, ?it/s]

In [22]:
# saving pictures
for i, region in tqdm(enumerate(region_datas), total=len(region_datas)):
    for column in region.columns[1:76]:
        plt.figure(figsize(15,7))
        region_datas[i][column].plot()
        region_datas[i]["{}_model".format(column)][8:].plot(color='r')
        plt.ylabel("{}, {}".format(region['oktmo'][0], column))
        plt.savefig('./pictures_new/{}_{}'.format(i, column))
        plt.close()    # close the figure window

  0%|          | 0/85 [00:00<?, ?it/s]

In [16]:
test = pd.read_csv('data/test.csv')
test['date'] = test['date'].apply(lambda x: pd.to_datetime(x, dayfirst=True))
test = test[['region', 'oktmo', 'okato', 'date']]
test_prev = test

In [17]:
columns = []
for column in region_datas[0].columns:
    if 'model' in column:
        columns.append(column)

In [18]:
for i in range(len(region_datas)):
    region_datas[i]['oktmo'] = region_datas[i]['oktmo'][0]

result = region_datas[0][-91:].copy()

for region in region_datas[1:]:
    result = pd.concat([result, region[-91:]])

result = result[['oktmo']+ columns]
result['date'] = result.index
result.columns = result.columns.str.replace('_model$', '')
result.columns = result.columns.str.replace('_x$', '')
result.columns = result.columns.str.replace('_y$', '')

for column in result.columns[1:-1]:
    result[column] = result[column].replace([np.inf, -np.inf], np.nan).fillna(0).apply(lambda x: round(x, 2) if x > 0 else 0)

result['oktmo'] = result['oktmo'].apply(int)

C:\Users\Strix\AppData\Local\Temp/ipykernel_23080/1657717950.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  result.columns = result.columns.str.replace('_model$', '')
C:\Users\Strix\AppData\Local\Temp/ipykernel_23080/1657717950.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  result.columns = result.columns.str.replace('_x$', '')
C:\Users\Strix\AppData\Local\Temp/ipykernel_23080/1657717950.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  result.columns = result.columns.str.replace('_y$', '')


In [19]:
for col in result.columns[4:]:
    result[col] = result[col].apply(lambda x: round(x * 1.01, 2) if type(x) == float else x)

In [20]:
test = test.merge(result, on=['date', 'oktmo'])

In [115]:
test['oktmo'].isnull().any().any()

False

In [21]:
test.to_csv('data/submission_final.csv', index=False)